In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
fifa_df = pd.read_csv("fifa21_raw_data.csv",low_memory=False)

In [3]:
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  object
 1   LongName          18979 non-null  object
 2   playerUrl         18979 non-null  object
 3   Nationality       18979 non-null  object
 4   Positions         18979 non-null  object
 5   Name              18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Team & Contract   18979 non-null  object
 10  ID                18979 non-null  int64 
 11  Height            18979 non-null  object
 12  Weight            18979 non-null  object
 13  foot              18979 non-null  object
 14  BOV               18979 non-null  int64 
 15  BP                18979 non-null  object
 16  Growth            18979 non-null  int64 
 17  Joined      

In [4]:
fifa_df.drop(["playerUrl","photoUrl"],axis=1,inplace=True)

## Converting Height(ft) to Height(inch)

In [5]:
import re

In [6]:
r = re.compile(r"(\d{1,2})'(\d{1,2})")

In [7]:
def in_inches(height):

    m = r.match(height)
    if m == None:
        return float("NaN")
    else:
        return int(m.group(1))*12+ float(m.group(2))

In [8]:
fifa_df["Height(inch)"] = fifa_df["Height"].apply(in_inches)

In [9]:
fifa_df[["Height","Height(inch)"]].head()

,Height,Height(inch)
0,"5'7""",67.0
1,"6'2""",74.0
2,"6'2""",74.0
3,"5'11""",71.0
4,"5'9""",69.0


## Converting K and M to its coresponding values

In [10]:
fifa_df["Value"].head()

0    €67.5M
1      €46M
2      €75M
3      €87M
4      €90M
Name: Value, dtype: object

In [11]:
df_value = fifa_df["Value"].str.strip("€").str.extract("(\d+\.?\d*)(K|M)").replace({'M':1000000,'K':1000})

df_value[0] = pd.to_numeric(df_value[0], errors='coerce')

fifa_df["Value"] = df_value.prod(axis=1)

In [12]:
fifa_df["Value"].head()

0    67500000.0
1    46000000.0
2    75000000.0
3    87000000.0
4    90000000.0
Name: Value, dtype: float64

In [13]:
fifa_df["Wage"].head()

0    €560K
1    €220K
2    €125K
3    €370K
4    €270K
Name: Wage, dtype: object

In [14]:
df_wage = fifa_df["Wage"].str.strip("€").str.extract("(\d+\.?\d*)(K|M)").replace({"M":1000000,"K":1000})

df_wage[0] = pd.to_numeric(df_wage[0], errors='coerce')

fifa_df["Wage"] = df_wage.prod(axis=1)

In [15]:
fifa_df["Wage"].head()

0    560000.0
1    220000.0
2    125000.0
3    370000.0
4    270000.0
Name: Wage, dtype: float64

In [16]:
fifa_df["Release Clause"].head()

0    €138.4M
1     €75.9M
2    €159.4M
3      €161M
4    €166.5M
Name: Release Clause, dtype: object

In [17]:
df_release = fifa_df["Release Clause"].str.strip("€").str.extract("(\d+\.?\d*)(K|M)").replace({"M":1000000,"K":1000})

df_release[0] = pd.to_numeric(df_release[0], errors='coerce')

fifa_df["Release Clause"] = df_release.prod(axis=1)

In [18]:
fifa_df["Release Clause"]

0        138400000.0
1         75900000.0
2        159400000.0
3        161000000.0
4        166500000.0
            ...     
18974        57000.0
18975       165000.0
18976        70000.0
18977       165000.0
18978       167000.0
Name: Release Clause, Length: 18979, dtype: float64

## Removing the STAR(★) characters

In [19]:
fifa_df["W/F"].head()

0    4 ★
1    4 ★
2    3 ★
3    5 ★
4    5 ★
Name: W/F, dtype: object

In [20]:
fifa_df["W/F"] = fifa_df["W/F"].apply(lambda x:x.split()[0])

In [21]:
fifa_df["W/F"].head()

0    4
1    4
2    3
3    5
4    5
Name: W/F, dtype: object

In [22]:
fifa_df["SM"].head()

0    4★
1    5★
2    1★
3    4★
4    5★
Name: SM, dtype: object

In [23]:
fifa_df["SM"] = fifa_df["SM"].str.strip("★")

In [24]:
fifa_df["SM"].head()

0    4
1    5
2    1
3    4
4    5
Name: SM, dtype: object

In [25]:
fifa_df['IR'].head()

0    5 ★
1    5 ★
2    3 ★
3    4 ★
4    5 ★
Name: IR, dtype: object

In [26]:
fifa_df["IR"] = fifa_df["IR"].apply(lambda x:x.split()[0])

In [27]:
fifa_df["IR"]

0        5
1        5
2        3
3        4
4        5
        ..
18974    1
18975    1
18976    1
18977    1
18978    1
Name: IR, Length: 18979, dtype: object

## Converting Date format

In [28]:
fifa_df["Joined"]

0         Jul 1, 2004
1        Jul 10, 2018
2        Jul 16, 2014
3        Aug 30, 2015
4         Aug 3, 2017
             ...     
18974     Aug 1, 2020
18975     Aug 1, 2020
18976    Jul 13, 2018
18977     Aug 1, 2020
18978     Jan 1, 2020
Name: Joined, Length: 18979, dtype: object

In [29]:
fifa_df["Joined"] = pd.to_datetime(fifa_df["Joined"],format="mixed")

In [30]:
fifa_df["Joined"].head()

0   2004-07-01
1   2018-07-10
2   2014-07-16
3   2015-08-30
4   2017-08-03
Name: Joined, dtype: datetime64[ns]

In [31]:
fifa_df["Loan Date End"].isnull().sum()   # Checking Null/NaN 

17966

In [32]:
fifa_df[~fifa_df["Loan Date End"].isnull()]["Loan Date End"]

205      Jun 30, 2021
250      Jun 30, 2021
257      Jun 30, 2021
299      Jun 30, 2021
305      Jun 30, 2021
             ...     
18497    Aug 31, 2021
18569    Dec 31, 2020
18580    Jun 30, 2021
18638    Dec 31, 2020
18696    Dec 31, 2020
Name: Loan Date End, Length: 1013, dtype: object

In [33]:
fifa_df["Loan Date End"] = pd.to_datetime(fifa_df["Loan Date End"],format="mixed")

In [34]:
fifa_df[~fifa_df["Loan Date End"].isnull()]["Loan Date End"]

205     2021-06-30
250     2021-06-30
257     2021-06-30
299     2021-06-30
305     2021-06-30
           ...    
18497   2021-08-31
18569   2020-12-31
18580   2021-06-30
18638   2020-12-31
18696   2020-12-31
Name: Loan Date End, Length: 1013, dtype: datetime64[ns]

## Removing new line characters (\n)

In [35]:
fifa_df[["Team & Contract", "Hits"]].head()

,Team & Contract,Hits
0,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,\n372
1,\n\n\n\nJuventus\n2018 ~ 2022\n\n,\n344
2,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,\n86
3,\n\n\n\nManchester City\n2015 ~ 2023\n\n,\n163
4,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,\n273


In [36]:
fifa_df["Team & Contract"] = fifa_df["Team & Contract"].apply(lambda x:x.strip("\n"))

fifa_df["Team & Contract"] = fifa_df["Team & Contract"].apply(lambda x:" ".join(x.split("\n")))

In [37]:
fifa_df["Team & Contract"].head()

0           FC Barcelona 2004 ~ 2021
1               Juventus 2018 ~ 2022
2        Atlético Madrid 2014 ~ 2023
3        Manchester City 2015 ~ 2023
4    Paris Saint-Germain 2017 ~ 2022
Name: Team & Contract, dtype: object

In [38]:
fifa_df["Hits"] = fifa_df["Hits"].str.strip("\n")

In [39]:
fifa_df["Hits"].head()

0    372
1    344
2     86
3    163
4    273
Name: Hits, dtype: object

In [40]:
fifa_df[["Team & Contract", "Hits"]].head()

,Team & Contract,Hits
0,FC Barcelona 2004 ~ 2021,372
1,Juventus 2018 ~ 2022,344
2,Atlético Madrid 2014 ~ 2023,86
3,Manchester City 2015 ~ 2023,163
4,Paris Saint-Germain 2017 ~ 2022,273


## Splitting into seperate Team and Contract

In [41]:
import re

In [42]:
fifa_df["Team & Contract"]

0                               FC Barcelona 2004 ~ 2021
1                                   Juventus 2018 ~ 2022
2                            Atlético Madrid 2014 ~ 2023
3                            Manchester City 2015 ~ 2023
4                        Paris Saint-Germain 2017 ~ 2022
                              ...                       
18974    Chongqing Dangdai Lifan FC SWM Team 2020 ~ 2020
18975                        Oldham Athletic 2020 ~ 2021
18976                             Wuhan Zall 2018 ~ 2022
18977                        Oldham Athletic 2020 ~ 2021
18978                           Club Bolívar 2020 ~ 2024
Name: Team & Contract, Length: 18979, dtype: object

In [43]:
def sep_team(team):

    m = re.findall(r"\D+",team)
    return(m[0])

In [44]:
fifa_df["Team"] = fifa_df["Team & Contract"].apply(sep_team)

In [45]:
fifa_df["Team"] = fifa_df["Team"].str.rstrip()    # To remove empty space at every end

In [46]:
fifa_df["Team"].head()

0           FC Barcelona
1               Juventus
2        Atlético Madrid
3        Manchester City
4    Paris Saint-Germain
Name: Team, dtype: object

In [47]:
def sep_contract(contract):

    p = re.findall(r"\d+",contract)
    return(" - ".join(p))

In [48]:
fifa_df["Contract"] = fifa_df["Team & Contract"].apply(sep_contract)

In [49]:
fifa_df["Contract"].head()

0    2004 - 2021
1    2018 - 2022
2    2014 - 2023
3    2015 - 2023
4    2017 - 2022
Name: Contract, dtype: object

In [50]:
fifa_df.head()

,LongName,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,ID,Height,...,PAC,SHO,PAS,DRI,DEF,PHY,Hits,Height(inch),Team,Contract
0,Lionel Messi,Argentina,RW ST CF,L. Messi,33,93,93,FC Barcelona 2004 ~ 2021,158023,"5'7""",...,85,92,91,95,38,65,372,67.0,FC Barcelona,2004 - 2021
1,C. Ronaldo dos Santos Aveiro,Portugal,ST LW,Cristiano Ronaldo,35,92,92,Juventus 2018 ~ 2022,20801,"6'2""",...,89,93,81,89,35,77,344,74.0,Juventus,2018 - 2022
2,Jan Oblak,Slovenia,GK,J. Oblak,27,91,93,Atlético Madrid 2014 ~ 2023,200389,"6'2""",...,87,92,78,90,52,90,86,74.0,Atlético Madrid,2014 - 2023
3,Kevin De Bruyne,Belgium,CAM CM,K. De Bruyne,29,91,91,Manchester City 2015 ~ 2023,192985,"5'11""",...,76,86,93,88,64,78,163,71.0,Manchester City,2015 - 2023
4,Neymar da Silva Santos Jr.,Brazil,LW CAM,Neymar Jr,28,91,91,Paris Saint-Germain 2017 ~ 2022,190871,"5'9""",...,91,85,86,94,36,59,273,69.0,Paris Saint-Germain,2017 - 2022


In [51]:
fifa_df.to_csv("Cleaned_Fifa.csv")